In [1]:
from dash import Dash, dcc, callback, Output, Input, html
import dash_bootstrap_components as dbc
import dash_vega_components as dvc
import altair as alt
#from vega_datasets import data
import pandas as pd

df = pd.read_csv("../data/raw/filtered_canada.csv")

new_df = df[(df['Type of corporation'] == 'Total all corporations') & 
            (df['Industry'] == 'Total all industries') & 
            (df["Size of enterprise"] == "Total all sizes") &
            (df["Unit of measure"] == "Number") &
            (df["Executive"] == "All officers\xa0") &
            (df["GEO"] != "Canada, total") &
            (df["GEO"] != "Unclassified province or territory")
].copy()
new_df = new_df.loc[:, ["REF_DATE", "Gender", "GEO", "VALUE"]]
new_df.head()

new_df.to_csv('../data/filtered/province_data.csv', index=False)
new_df["GEO"].unique()

array(['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories',
       'Nunavut'], dtype=object)

In [2]:
df = pd.read_csv('../data/filtered/province_data.csv')



In [3]:

# Initiatlize the app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = dbc.Container([
    dvc.Vega(id='pie-chart', spec={}),
    html.P(html.Br()),
    html.Label('Select a Year'),
    dcc.Dropdown(id='year', options=df["REF_DATE"].unique().tolist(), value = 2016),
    html.Label('Select a Province'),
    dcc.Dropdown(id='prov', options = df["GEO"].unique().tolist(), value ='British Columbia')
])

# Server side callbacks/reactivity
@callback(
    Output('pie-chart', 'spec'),
    Input('year', 'value'),
    Input('prov', 'value'),
)

def create_chart(year, prov):

    # Issue is with industry and year datatype - not recognised by pandas, so it returns empty df (filtered_df)
    filtered_df = df[(df["GEO"] == prov) & (df["REF_DATE"] == year)]

    
    chart = alt.Chart(filtered_df).mark_arc().encode(
        theta="VALUE",
        color="Gender", 
        tooltip=['Gender:N', 'VALUE:Q']).properties(
        title="Number of Women vs Men in Executive Positions"
)


    return chart.to_dict()

# Run the app/dashboard ##XXXXyear and province for all industries
app.run()